<a href="https://colab.research.google.com/github/AlcidesChewe/TP_MachineLearning/blob/main/TP_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install numpy-stl

In [13]:
!git clone https://github.com/dimatura/binvox-rw-py.git

Cloning into 'binvox-rw-py'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Receiving objects: 100% (66/66), 76.84 KiB | 624.00 KiB/s, done.
Resolving deltas: 100% (33/33), done.


In [18]:
!pip install trimesh[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.4/685.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.4/229.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
  Created wheel for pycollada: filename=pycollada-0.7.2-py3-none-any.whl size=127016 sha256=6795ba187a4e7d9ae835ec3c05b9155829c909364f7a569c8b4e374e4ba7b7c1
  Stored in directory: /root/.cache/pip/wheels/d5/ba/33/1e99a7e7defd1d77f0210e7

In [14]:
import sys
sys.path.append('/content/binvox-rw-py')

In [22]:
import numpy as np
from stl import mesh
import numpy as np
import trimesh
import binvox_rw

In [5]:
# Cargar el modelo 3D de la taza
your_mesh = mesh.Mesh.from_file('beker1.stl')

# Obtener las dimensiones máximas y mínimas en cada eje
min_x, min_y, min_z = your_mesh.min_[:]
max_x, max_y, max_z = your_mesh.max_[:]

# Calcular las dimensiones aproximadas
width = max_x - min_x
height = max_z - min_z
depth = max_y - min_y

print(f"Dimensiones Aproximadas de la Taza:")
print(f"Ancho (X): {width} unidades")
print(f"Alto (Z): {height} unidades")
print(f"Profundidad (Y): {depth} unidades")


Dimensiones Aproximadas de la Taza:
Ancho (X): 46.16403579711914 unidades
Alto (Z): 45.0 unidades
Profundidad (Y): 46.15821838378906 unidades


In [16]:
# 1. Lectura del archivo STL
your_mesh = mesh.Mesh.from_file('beker1.stl')

# 2. Normalización
max_dimensions = np.max(your_mesh.vectors, axis=0) - np.min(your_mesh.vectors, axis=0)
scale_factor = 50 / max_dimensions  # Escalar a 50 unidades
your_mesh.vectors *= scale_factor

# 3. Almacenamiento
your_mesh.save('beker1_normalized.stl')

In [20]:
# Cargar el archivo STL
mesh = trimesh.load_mesh('beker1_normalized.stl')

# Convertir a voxels
voxel_grid = mesh.voxelized(pitch=0.5)  # puedes ajustar el valor de 'pitch' según la resolución deseada

# Visualizar el resultado (opcional)
voxel_grid.show()

with open('beker1_normalized.binvox', 'wb') as f:
    voxel_grid.export(f, file_type='binvox')

In [25]:
from stl import mesh as stl_mesh
# 1. Lectura del archivo STL normalizado
your_mesh = stl_mesh.Mesh.from_file('beker1_normalized.stl')

# 2. Conversión a Voxels (Asumiendo que ya tienes el archivo 'beker1_normalized.binvox')
with open('beker1_normalized.binvox', 'rb') as f:
    model = binvox_rw.read_as_3d_array(f)

# 3. Reducción de Dimensionalidad
downsampled_voxels = model.data[::2, ::2, ::2]

# 4. Normalización
normalized_voxels = downsampled_voxels.astype(np.float32)


# **Simulacion de rotura**

In [10]:
# Load the mesh using numpy-stl
your_mesh = mesh.Mesh.from_file('beker1.stl')

# Determine the z-coordinates of all vertices
z_coords = your_mesh.vectors[:,:,2]

# Find the median z-coordinate to split the taza approximately in half
median_z = np.median(z_coords)

# Remove triangles whose three vertices are below the median z-coordinate
mask = np.all(z_coords < median_z, axis=1)
your_mesh.vectors[mask] = np.zeros((3, 3))

# Save the modified mesh to a new file
your_mesh.save('beker1_roto.stl')

# **Parte rota**


In [11]:
# Load the mesh using numpy-stl
your_mesh = mesh.Mesh.from_file('beker1.stl')

# Determine the z-coordinates of all vertices
z_coords = your_mesh.vectors[:,:,2]

# Find the median z-coordinate to split the taza approximately in half
median_z = np.median(z_coords)

# Remove triangles whose three vertices are above the median z-coordinate
mask = np.all(z_coords > median_z, axis=1)
your_mesh.vectors[mask] = np.zeros((3, 3))

# Save the modified mesh to a new file
your_mesh.save('beker1_parte_superior.stl')
